In [115]:
import numpy as np
runif=np.random.uniform
from scipy.special import gamma
import time
from collections import deque

i copied the professor matlab code into python.


This generates the arrivals time and the service times

In [132]:
###here i just write in python the matlab script from the prof

ntry=10
n=100000
mTv=[]
ratioTv=[]
mXv=[]
ratioXv=[]
telapsedv=[]
rhosimv=[]
for i in range(ntry):
    #start timer
    start=time.time()
    
    #arrival time
    T0=1
    #average Y
    Ymed=10
    #prob q
    q=3/5
    #average time arrival
    ET=T0+(1-q)*Ymed
    #variance of time arrival
    ET2=T0**2+(1-q)*(2*Ymed**2+2*T0*Ymed)
    #
    CT=np.sqrt(ET2-ET**2)/ET
    #random vector
    v=np.round(q-runif(0,1,n))
    #vector of generated arrival times
    Tv=T0-(1-v)*np.log(runif(0,1,n))*Ymed
    #its mean
    mT=np.mean(Tv)
    #its std
    sT=np.std(Tv)
    # std over mean
    cT = sT/mT
    #confidence interval of T
    ICT = 1.96*sT/np.sqrt(n)
    # compute ratio
    ratioT=ICT/mT
    # save to the vector
    mTv.append(mT)
    # save to the vector
    ratioTv.append(ratioT)

    #processing time expected value
    EX=99
    #weibull distro parameters
    alfa=1/2
    beta=EX/gamma(1+1/alfa)
    EX2=beta**2*gamma(1+2/alfa)
    CX=np.sqrt(EX2-EX**2)/EX
    #generate vector of processing time 
    Xv=np.maximum(1,np.minimum(100*EX, np.round(beta*(-np.log(runif(0,1,n)))**(1/alfa))))
    #compute metrics on the processing times
    mX=np.mean(Xv)
    sX=np.std(Xv)
    cX=sX/mX
    ICX=1.96*sX/np.sqrt(n)
    ratioX=ICX/mX
    mXv.append(mX)
    ratioXv.append(ratioX)
    
    #utilization coeff
    
    #N server
    Nse=20
    rho=EX/(Nse*ET)
    rhosim=mX/Nse/mT
    rhosimv.append(rhosim)
    
    end = time.time()
    telapsed= end - start
    telapsedv.append(telapsed)
    
# print(f'estimated average inter arrival time are \n{mTv}\n')
# print(f'ratio of 95% CI to estimated average of interarrival times are \n{ratioTv}\n')
# print(f'estimated average service time are \n{mXv}\n')
# print(f'ratio of 95% CI of estimated average of service time \n{ratioXv}\n')
# print(f'estimated utilization coefficient \n{rhosimv}\n')
# print('')
# print(f'average computation time {1000*np.mean(telapsedv)} ms')

with the generated times i run a POD

In [136]:
#N servers
N=20
#start with empty server
server= [deque() for i in range(N)]
#memory to know how many task in queue for each server
memory= [0]*N
#arrival times of tasks (as cumulative sum of arrival times)
task_arrival=np.cumsum(mTv)
#service times
service_times=mXv
#POD parameter
d=3

for time, serviceT in zip(task_arrival, service_times):
    print(f'we are at time {time}')
    
    #check if at time=time there are task that are done,
    #if they are done, delete them from the queue
    for i in range(len(server)):
        try:
            if server[i][0]<time:
                server[i].popleft()
        except:
            continue
    
    #POD, select 3 servers and check how many task in queue
    for i in range(d):
        rnd_server=np.random.choice(list(range(N)))  
        memory[rnd_server]=len(server[rnd_server]) 
        
    #save minimun length value
    min_value = min(memory)
    #save the index of the server with minimum length queue
    to_choose=[i for i, x in enumerate(memory) if x == min_value]
    #select randomly over server with queue with min len over the d sever extracted
    selected_server=np.random.choice(to_choose)
    
    #update server, with new task saved as with time T the task will be over (current time+service time)
    server[selected_server].append(serviceT+time)
    #update memory
    memory[selected_server]+=1


we are at time 9.97739198285275
we are at time 19.970540342328015
we are at time 29.955798168407295
we are at time 39.98576411815362
we are at time 49.973623835517685
we are at time 60.01826914956663
we are at time 70.0491626210309
we are at time 80.0208349367975
we are at time 90.05214482608605
we are at time 100.08871916217406


This is the state of the servers at the end of the simulation.

each servers has a queue of task, each task of the queue has the time at which it will be over 

In [137]:
server

[deque([120.192510342328]),
 deque([159.23333914956663]),
 deque([]),
 deque([149.2821938355177]),
 deque([]),
 deque([139.2814341181536]),
 deque([188.37315482608605]),
 deque([]),
 deque([]),
 deque([]),
 deque([198.06724916217405]),
 deque([]),
 deque([]),
 deque([170.5045226210309]),
 deque([]),
 deque([]),
 deque([129.3675381684073]),
 deque([108.98114198285275]),
 deque([180.3613249367975]),
 deque([])]

The memory keeps track of the length of the queues

In [138]:
memory

[1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0]

### just notes and garbage, you can ignore this

In [20]:
#this is the function of the pod
def POD(d,N,server):
    #i assume that we start not knowing the queue in all server
    memory=[0]*N
    #here i choose d random servers and save their queue to the memory
    for i in range(d):
        #choice a server randomly over a list of N
        rnd_server=random.choice(list(range(N)))
        #save it's queue length to memory
        memory[rnd_server]=server[rnd_server]
    #save minimun length value
    min_value = min(memory)
    #save the index of the server with minimum length queue
    to_choose=[i for i, x in enumerate(memory) if x == min_value]
    #select randomly over server with queue with min len over the d sever extracted
    selected_server=random.choice(to_choose)
    #update memory, now that the chosen server has 1 more user in queue
    memory[selected_server]+=1
    return selected_server